In [3]:
import pandas as pd
import numpy as np
import scipy as sc
from scipy import signal

import matplotlib.pyplot as plt
import h5py
import obspy
from obspy.signal.filter import envelope
from obspy.clients.fdsn import Client
from tqdm import tqdm
from glob import glob
import tsfel
import random


client = Client('IRIS')

import os
import pickle

import matplotlib.pyplot as plt
import numpy as np

from scatseisnet import ScatteringNetwork
import matplotlib.dates as mdates


import time

from scipy.signal import butter, filtfilt
from scipy.signal import butter, lfilter

#%config InlineBackend.figure_format = "svg"

#from Feature_Extraction import compute_hibert




# collecting waveform data and corresponding catalog.
comcat_file_name = "/data/whd01/yiyu_data/PNWML/comcat_waveforms.hdf5"
comcat_csv_file = "/data/whd01/yiyu_data/PNWML/comcat_metadata.csv"




cat_comcat = pd.read_csv(comcat_csv_file)


## According to paper the waveforms are detrended but not filtered. (https://seismica.library.mcgill.ca/article/view/368/868)
## According to tutorial  - we need to detrend and bandpass filter, Since we highpass filtered the data in case of other features we are going to do the same in this case as well. 


def extract_waveforms(cat, file_name):
    
    st = []
    cat_trace = cat['trace_name'].values
    comcat_id = []
    serial_no = []
    for i in range(0,50000):

            
 
        f = h5py.File(file_name, 'r')
            
            
        bucket = cat['trace_name'].values[i].split('$')[0]
        ind = int(cat['trace_name'].values[i].split('$')[1].split(',')[0])
        # ENZ

        
        if np.sum(f['/data/'+bucket][ind, 0, :15000]) != 0:
            st.append(f['/data/'+bucket][ind, :3, :15000])  
            serial_no.append(i)
            comcat_id.append(cat['event_id'].values[i])
        
    return st, comcat_id, serial_no






def cosine_taper_bandpass_filter(data, sampling_rate):
    # Define the percentage of tapering (10%)
    taper_percentage = 0.10

    # Create a cosine taper window
    taper_length = int(len(data) * taper_percentage)
    taper = 0.5 * (1 - np.cos(2 * np.pi * np.arange(taper_length) / (taper_length - 1)))

    # Apply the taper to the beginning and end of the data
    data[:taper_length] *= taper
    data[-taper_length:] *= taper[::-1]

    # Define the bandpass filter parameters
    lowcut = 1.0  # Lower cutoff frequency in Hz
    highcut = 10.0  # Upper cutoff frequency in Hz

    # Normalize the cutoff frequencies
    lowcut /= (0.5 * sampling_rate)
    highcut /= (0.5 * sampling_rate)

    # Design the Butterworth bandpass filter
    b, a = butter(4, [lowcut, highcut], btype='band')

    # Apply the bandpass filter to the data
    filtered_data = lfilter(b, a, data)

    return filtered_data



# Function to compute the absolute Fourier transform of an array
def compute_absolute_fft(arr):
    fft_result = np.fft.fft(arr)  # Compute the FFT
    absolute_fft = np.abs(fft_result)  # Compute the absolute value of the FFT
    return absolute_fft








## Comcat


st_comcat, comcat_id, serial_no = extract_waveforms(cat_comcat, comcat_file_name)

fs = 100  # Sampling frequency in Hz
cutoff = 1  # Cutoff frequency in Hz


data = np.array(st_comcat)[:,2,4000:9000]
filtered_data = cosine_taper_bandpass_filter(data, fs)
        
        
        
        
# Compute the absolute Fourier transform for each array in the array of arrays
absolute_fft_results = [compute_absolute_fft(arr) for arr in filtered_data]

        
 
"""" 
np.savez(
    "/home/ak287/PNW_ML_Classification/extracted_features/scatnet_features/p_10_40/comcat_scattering_coefficients_part1.npz",
    order_1=scattering_coefficients[0],
    order_2=scattering_coefficients[1],
    event_ids = comcat_id,
    serial_ids = serial_no,
    duration = dur_scatnet_comcat
)

"""


KeyboardInterrupt: 

In [3]:
filtered_data[0]

array([[  0.00000000e+00,   3.86222617e-05,   4.01479269e-04, ...,
         -3.17822239e+00,  -2.83488494e+00,  -2.57481661e+00],
       [  0.00000000e+00,  -1.05515315e-04,  -1.09758337e-03, ...,
          2.17008437e+00,   2.10032630e+00,   2.06757812e+00],
       [  0.00000000e+00,  -6.37148276e-06,  -6.72090214e-05, ...,
          5.99979236e+00,   6.03626255e+00,   6.05710285e+00]])

In [30]:
# importing the dependencies. 
import pandas as pd
import numpy as np
import scipy as sc
from scipy import signal

import matplotlib.pyplot as plt
import h5py
import obspy
from obspy.signal.filter import envelope
from obspy.clients.fdsn import Client
from tqdm import tqdm
from glob import glob
import tsfel
import random
from datetime import timedelta
import calendar
from tsfel import time_series_features_extractor











#%config InlineBackend.figure_format = "png"

#from Feature_Extraction import compute_hibert

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# displaying all columns from pandas dataframe
# Set display options to show all columns
pd.set_option('display.max_columns', None)

import time


        
              
              

              
exotic_file_name = "/data/whd01/yiyu_data/PNWML/exotic_waveforms.hdf5"
exotic_csv_file = pd.read_csv("/data/whd01/yiyu_data/PNWML/exotic_metadata.csv")
f = h5py.File(exotic_file_name, 'r')

buckets = [exotic_csv_file['trace_name'].values[i].split('$')[0] for i in range(len(exotic_csv_file))]
indices = [int(exotic_csv_file['trace_name'].values[i].split('$')[1].split(',')[0]) for i in range(len(exotic_csv_file))]

source = exotic_csv_file['source_type'].values


buck_su = np.array(buckets)[np.where(source == 'surface event')[0]]
ind_su = np.array(indices)[np.where(source == 'surface event')[0]]
su_ids = exotic_csv_file[exotic_csv_file['source_type'] == 'surface event']['event_id'].values

data_su = []
serial_no = []
for i in tqdm(range(len(buck_su))):
     data_su.append(f['/data/'+buck_su[i]][ind_su[i], 2, 6000:11000])
     serial_no.append(i)
        
        
data_su = np.array(data_su)



buck_th = np.array(buckets)[np.where(source == 'thunder')[0]]
ind_th = np.array(indices)[np.where(source == 'thunder')[0]]

data_th = []
for i in tqdm(range(len(buck_th))):
     data_th.append(f['/data/'+buck_th[i]][ind_th[i], 2, 6000:11000])
        
        
data_th = np.array(data_th)


buck_sb = np.array(buckets)[np.where(source == 'sonic boom')[0]]
ind_sb = np.array(indices)[np.where(source == 'sonic boom')[0]]

data_sb = []
for i in tqdm(range(len(buck_sb))):
     data_sb.append(f['/data/'+buck_sb[i]][ind_sb[i], 2, 6000:11000])
        
        
data_sb = np.array(data_sb) 
              
  
              
              
       
            
        
   
import numpy as np

def apply_cosine_taper(arrays, taper_percent=10):
    tapered_arrays = []
    
    #print(arrays.shape)
    num_samples = arrays.shape[1]  # Assuming each sub-array has the same length
    
    for array in arrays:
        

        taper_length = int(num_samples * taper_percent / 100)
        taper_window = np.hanning(2 * taper_length)
        
     
        tapered_array = array.copy()
        tapered_array[:taper_length] = tapered_array[:taper_length] * taper_window[:taper_length]
        tapered_array[-taper_length:] = tapered_array[-taper_length:] * taper_window[taper_length:]
        
        tapered_arrays.append(tapered_array)
    
    return np.array(tapered_arrays)


              
              
              
              
              
              
              

import numpy as np
import scipy.signal as signal

def butterworth_filter(arrays, lowcut, highcut, fs, num_corners, filter_type='bandpass'):
    """
    Apply a Butterworth filter (bandpass, highpass, or lowpass) to each array in an array of arrays.

    Parameters:
        arrays (list of numpy arrays): List of arrays to be filtered.
        lowcut (float): Lower cutoff frequency in Hz.
        highcut (float): Upper cutoff frequency in Hz.
        fs (float): Sampling frequency in Hz.
        num_corners (int): Number of corners (filter order).
        filter_type (str, optional): Type of filter ('bandpass', 'highpass', or 'lowpass'). Default is 'bandpass'.

    Returns:
        list of numpy arrays: List of filtered arrays.
    """
    filtered_arrays = []
    for data in arrays:
        # Normalize the frequency values to Nyquist frequency (0.5*fs)
        lowcut_norm = lowcut / (0.5 * fs)
        highcut_norm = highcut / (0.5 * fs)

        # Design the Butterworth filter based on the filter type
        if filter_type == 'bandpass':
            b, a = signal.butter(num_corners, [lowcut_norm, highcut_norm], btype='band')
        elif filter_type == 'highpass':
            b, a = signal.butter(num_corners, lowcut_norm, btype='high')
        elif filter_type == 'lowpass':
            b, a = signal.butter(num_corners, highcut_norm, btype='low')
        else:
            raise ValueError("Invalid filter_type. Use 'bandpass', 'highpass', or 'lowpass'.")

        # Apply the filter to the data using lfilter
        filtered_data = signal.lfilter(b, a, data)

        filtered_arrays.append(filtered_data)

    return filtered_arrays

    
# Function to compute the absolute Fourier transform of an array
def compute_absolute_fft(arr):
    fft_result = np.fft.fft(arr)  # Compute the FFT
    absolute_fft = np.abs(fft_result)  # Compute the absolute value of the FFT
    return absolute_fft



              
tapered_su = apply_cosine_taper(data_su, taper_percent = 10) 
tapered_th = apply_cosine_taper(data_th, taper_percent = 10)  
tapered_sb = apply_cosine_taper(data_sb, taper_percent = 10)  

filtered_su = np.array(butterworth_filter(tapered_su, 1, 10, 100, 4, 'bandpass'))
filtered_th = np.array(butterworth_filter(tapered_th, 1, 10, 100, 4, 'bandpass'))
filtered_sb = np.array(butterworth_filter(tapered_sb, 1, 10, 100, 4, 'bandpass'))

              


su_Z = filtered_su
th_Z = filtered_th
sb_Z = filtered_sb


              
# Normalizing the data.              

su_Z = su_Z/np.max(abs(su_Z), axis = 1)[:, np.newaxis]
th_Z = th_Z/np.max(abs(th_Z), axis = 1)[:, np.newaxis]
sb_Z = sb_Z/np.max(abs(sb_Z), axis = 1)[:, np.newaxis]

              
    
    
a = time.time()      
# Compute the absolute Fourier transform for each array in the array of arrays
absolute_fft_results = [compute_absolute_fft(arr) for arr in su_Z]

dur_scatnet_comcat = time.time() - a



    
np.savez(
    "/home/ak287/PNW_ML_Classification/extracted_features/afs_features/10_40_p/exotic_afs_features.npz",
    order_1=  absolute_fft_results,
    event_ids = su_ids,
    serial_ids = serial_no,
    duration = dur_scatnet_comcat
)           
              
       
              
              
              
              

100%|██████████| 206/206 [00:00<00:00, 3543.26it/s]


In [31]:
store_file = "/home/ak287/PNW_ML_Classification/extracted_features/afs_features/10_40_p/exotic_afs_features.npz"
data = np.load(store_file, allow_pickle= True)
order_1 = data["order_1"]
serial_ids = data['serial_ids']
dur_exotic = data["duration"]




exotic_features = pd.DataFrame(order_1)
exotic_metadata = exotic_csv_file.iloc[serial_ids]


exotic_features.index = serial_ids
exotic_all = pd.concat([exotic_features, exotic_metadata], axis = 1)

In [32]:
exotic_features

0         1         2         3         4         5         6     \
0     0.001734  0.001753  0.001935  0.002561  0.003739  0.005420  0.008344   
1     0.001716  0.002019  0.003033  0.005282  0.009724  0.015806  0.028821   
2     0.001369  0.001655  0.002722  0.005391  0.010600  0.018337  0.028863   
3     0.000225  0.000232  0.000205  0.000276  0.000818  0.001958  0.003869   
4     0.002968  0.003034  0.003521  0.004993  0.008067  0.011873  0.013936   
...        ...       ...       ...       ...       ...       ...       ...   
8907  0.056313  0.056336  0.056490  0.056979  0.057907  0.059019  0.060122   
8908  0.002484  0.002485  0.002488  0.002491  0.002501  0.002488  0.002394   
8909  0.008455  0.008473  0.008526  0.008616  0.008726  0.008791  0.008882   
8910  0.001623  0.001624  0.001627  0.001630  0.001642  0.001651  0.001670   
8911  0.004320  0.004319  0.004241  0.003891  0.003278  0.002260  0.001487   

          7         8         9         10        11        12        13    \
0     0.010546  0.007191  0.038737  0.024000  0.021983  0.051476  0.053702   
1     0.021006  0.016537  0.084730  0.085812  0.124059  0.252449  0.191456   
2     0.025467  0.037611  0.025145  0.013803  0.166667  0.172646  0.116996   
3     0.002853  0.014350  0.015531  0.013713  0.039299  0.023270  0.033612   
4     0.024084  0.006977  0.017886  0.086178  0.079757  0.112627  0.149188   
...        ...       ...       ...       ...       ...       ...       ...   
8907  0.060089  0.064230  0.065938  0.057353  0.042656  0.067579  0.079491   
8908  0.002338  0.003253  0.004396  0.002922  0.002676  0.004700  0.009398   
8909  0.007888  0.011710  0.016717  0.010029  0.003033  0.023565  0.029077   
8910  0.001691  0.001807  0.002295  0.000643  0.000516  0.003441  0.004022   
8911  0.001617  0.000193  0.005431  0.007816  0.007301  0.013122  0.003325   

          14        15        16        17        18        19        20    \
0     0.045124  0.051336  0.037289  0.050687  0.105760  0.198228  0.264333   
1     0.217045  0.162169  0.610425  0.384232  0.124105  0.476696  1.371510   
2     0.224056  0.162298  0.168760  0.207747  0.229622  0.283065  0.264938   
3     0.071691  0.001219  0.038379  0.055929  0.134043  0.014785  0.068278   
4     0.115265  0.227954  0.355703  0.056047  0.204790  0.048497  0.078008   
...        ...       ...       ...       ...       ...       ...       ...   
8907  0.024051  0.068509  0.099173  0.156994  0.118358  0.117281  0.149115   
8908  0.011108  0.020821  0.010333  0.035533  0.065298  0.105692  0.051640   
8909  0.041011  0.075964  0.035629  0.138842  0.273875  0.406014  0.137508   
8910  0.009185  0.011044  0.009185  0.004927  0.023634  0.008226  0.012937   
8911  0.016159  0.023481  0.001121  0.040050  0.042948  0.010098  0.011965   

          21        22        23        24        25        26        27    \
0     0.447190  0.310237  0.452297  0.466262  0.476388  0.363872  0.243408   
1     1.792984  0.698533  1.788929  1.380152  0.814733  2.156392  0.994020   
2     0.362041  0.564169  0.452185  0.459020  0.397771  0.548773  0.841930   
3     0.170523  0.272765  0.113660  0.415239  0.244492  0.388372  0.412666   
4     0.212753  0.241832  0.305229  0.717209  0.429722  0.331325  0.873740   
...        ...       ...       ...       ...       ...       ...       ...   
8907  0.200556  0.254485  0.166319  0.383699  0.530110  0.541192  1.988639   
8908  0.062960  0.025428  0.076890  0.183628  0.175076  0.065159  0.091621   
8909  0.271443  0.333132  0.277992  0.506750  0.199669  0.305147  0.180855   
8910  0.035124  0.027996  0.036122  0.123775  0.024463  0.063791  0.127116   
8911  0.099676  0.052811  0.101246  0.045585  0.133073  0.196931  0.429968   

          28        29        30        31        32        33        34    \
0     0.163615  0.656322  0.605337  1.429103  1.374697  1.911574  2.217638   
1     1.481366  1.309384  1.927909  0.516070  2.709385  1.561587  1.205266   
2     0.965540  1.356092  1.